In [19]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD
import os
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from random import shuffle

In [20]:
#spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.11.임시_통합DB.xlsx')
data = pd.read_excel('../최종DB관리/Session_Detail_DB/Session_train_totalv2.xlsx')

In [21]:
ids=data[data["Spot"]=='구리코 네온'].Session_id
check = 0
for i in ids:
    sums=data[data["Session_id"]==i]["Spot"].isin(['도톤보리']).sum()
    if sums == 0:
        check += 1
        data.loc[(data["Session_id"]==i)&(data["Spot"]=="구리코 네온"),"Spot"]="도톤보리"
        print(check)
    else:
        data.loc[(data["Session_id"]==i)&(data["Spot"]=="구리코 네온"),"Spot"]="삭제"

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [22]:
data2=data[data["Spot"]!='삭제']

In [23]:
spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.13.최종_통합DB2.xlsx')

In [24]:
spot["city_sub"]=spot.city +'_' + spot.sublocality

In [25]:
lab = LabelEncoder()
spot["city_sub_encoder"] = lab.fit_transform(spot["city_sub"])

In [26]:
data_spot = data2.merge(spot[["ID2","Spot","Tag_encoder",'City_encoder','city_sub_encoder',"State_encoder"]],on="Spot",how="left")

In [27]:
# session_length 1 인거 삭제
lst_1=list(data_spot.groupby("Session_id").size().index[np.where(data_spot.groupby('Session_id').size()==1)[0]])
data_spot = data_spot[~data_spot.Session_id.isin(lst_1)]

In [33]:
lst = []
for i in range(data_spot.Session_id.nunique()):
    c=np.random.randint(0,5)
    lst.append(c)

In [35]:
data_spot.Session_id.nunique()

3837

In [36]:
num = pd.DataFrame(columns=["Session_id","k"])
num["Session_id"] = data_spot.Session_id.unique()
num["k"] = lst

In [37]:
data_spot = data_spot.merge(num,on="Session_id",how="inner")

# 데이터 내보낸거 불러서하기

In [15]:
data_spot = pd.read_excel('data_spot_5_fold.xlsx')

In [39]:
def recommendations(ids,top ,cosine_sim_data):
    
    score_lst = cosine_sim_data[ids].sort_values(ascending=False)[1:top+1].values.tolist()  
    count_0=score_lst.count(0.0) # 0 점인 애들은 랜덤 추천
    if count_0 != 0:
        rec_lst_1 = list(cosine_sim_data[ids].sort_values(ascending=False)[1:top+1-count_0].index)
        random_lst = list(cosine_sim_data[ids].sort_values(ascending=False)[top+1-count_0:].index)
        shuffle(random_lst)
        rec_lst = rec_lst_1 + random_lst[:count_0]
    else:
        rec_lst = list(cosine_sim_data[ids].sort_values(ascending=False)[1:top+1].index)
    return rec_lst,score_lst

In [40]:
def evaluation(test,top,cosine_sim):
    recall = 0.0
    mrr = 0.0
    count = 0
    lst = []
    for idx in range(len(test)-1):
        sess_id = test["Session_id"].values[idx]
        id2 = test["new_id"].values[idx]
        #print(sess_id,id2)
        if sess_id == test["Session_id"].values[idx+1]:
            top_lst,score_lst = recommendations(id2,top=top,cosine_sim_data=cosine_sim)
            answer = test["new_id"].values[idx+1]
            lst += top_lst
            count += 1
            try:
                rank = top_lst.index(answer)
                recall += 1.0
                mrr += 1/(rank+1)
            except:
                continue
    return recall/count,mrr/count,len(set(lst))

In [41]:
result = pd.DataFrame(columns=["k","top","recall","mrr","unique_lst"])

new_id = pd.DataFrame(columns=["Spot","new_id"])
new_id["Spot"] = data_spot.Spot.unique()
new_id["new_id"] = range(data_spot.Spot.nunique())
data_spot = data_spot.merge(new_id,on="Spot")
data_spot = data_spot.sort_values(["Session_id","Session_num"])

for k in range(5):
    
    train = data_spot[data_spot["k"]!=k]
    test = data_spot[data_spot["k"]==k]
    
    
    datas = pd.DataFrame(columns=["Session_id","spots"])
    datas['Session_id'] = train.groupby('Session_id').new_id.apply(list).index
    datas['spots'] = train.groupby('Session_id').new_id.apply(list).values
    score = np.zeros((len(datas),1201)) # sess x item
    train_spot_lst = train.groupby('Session_id').new_id.apply(list).values.tolist()
    for i,j in enumerate(train_spot_lst):
        for s in j:
            #if score[i,s]==0:
            score[i,s] = 1 # binary matrix 구성
    score_t=score.T
    cosine_sim = cosine_similarity(score_t, score_t) # item 기준 cosine similariy
    for i in range(1201):
        cosine_sim[i,i]=1.0
    cosine_sim_data=pd.DataFrame(cosine_sim)
    for t in [5,10,15,20]:
        recall,mrr,unique_lst=evaluation(test,t,cosine_sim_data)
        mini=[k,t,recall,mrr,unique_lst]
        result_mini = pd.Series(mini,index=["k","top","recall","mrr","unique_lst"])
        result=result.append(result_mini,ignore_index=True)
        print(mini)

[0, 5, 0.32723449001051524, 0.19591657903960855, 940]
[0, 10, 0.45951629863301785, 0.21386885984677803, 1097]
[0, 15, 0.5331230283911672, 0.21960458223865265, 1151]
[0, 20, 0.5890641430073607, 0.2227838872773979, 1186]
[1, 5, 0.3475444096133751, 0.20727621037965993, 934]
[1, 10, 0.47732497387669803, 0.224725663863595, 1094]
[1, 15, 0.5479623824451411, 0.2302431801334627, 1153]
[1, 20, 0.5951933124346918, 0.2329136130445853, 1190]
[2, 5, 0.36017453391511306, 0.2075003305566588, 932]
[2, 10, 0.4884966283220944, 0.22497512954672813, 1064]
[2, 15, 0.5583101943673145, 0.23042339343410342, 1151]
[2, 20, 0.6112653708845697, 0.2334213743094389, 1178]
[3, 5, 0.33655172413793105, 0.19728078817734132, 952]
[3, 10, 0.46798029556650245, 0.2151498944405349, 1096]
[3, 15, 0.5548768472906404, 0.2220370729434775, 1161]
[3, 20, 0.6065024630541872, 0.22508554266409359, 1192]
[4, 5, 0.3475484136794396, 0.2047932976239541, 918]
[4, 10, 0.48702101359703337, 0.22353475451435903, 1072]
[4, 15, 0.5572723526988

In [42]:
result.to_excel('5_fold_base_line_final.xlsx')

In [43]:
print("recall mean",result.groupby('top').recall.apply(np.mean))
print("recall std",result.groupby('top').recall.apply(np.std))
print('--------------------')
print("mrr mean",result.groupby('top').mrr.apply(np.mean))
print("mrr std",result.groupby('top').mrr.apply(np.std))
print('--------------------')
print("unique_lst mean",result.groupby('top').unique_lst.apply(np.mean))
print("unique_lst std",result.groupby('top').unique_lst.apply(np.std))

recall mean top
5.0     0.343811
10.0    0.476068
15.0    0.550309
20.0    0.601831
Name: recall, dtype: float64
recall std top
5.0     0.011164
10.0    0.011094
15.0    0.009319
20.0    0.008318
Name: recall, dtype: float64
--------------------
mrr mean top
5.0     0.202553
10.0    0.220451
15.0    0.226220
20.0    0.229165
Name: mrr, dtype: float64
mrr std top
5.0     0.004973
10.0    0.004892
15.0    0.004511
20.0    0.004372
Name: mrr, dtype: float64
--------------------
unique_lst mean top
5.0      935.2
10.0    1084.6
15.0    1152.8
20.0    1186.8
Name: unique_lst, dtype: float64
unique_lst std top
5.0     11.070682
10.0    13.821722
15.0     4.400000
20.0     4.833218
Name: unique_lst, dtype: float64


In [ ]:
result.groupby('k').recall.apply(np.mean)

In [44]:
data_spot.to_excel('data_spot_5_fold.xlsx')

In [50]:
cc=pd.read_csv('D:/2019.01.04~여행앱개발/modeling/yoochoose-data/yoochoose-clicks.csv')

C:\Users\LeeWonSeok\Anaconda3\envs\for_tensor\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
data = pd.DataFrame(cc.values,columns=["Session_id","timestamp","item_id","category"])

In [54]:
data.item_id.nunique()

52739

In [45]:
data_spot

,Session_id2,Session_id,Session_num,Start_Date,End_Date,month,Long,Date_info,percent,Spot,ID2,Tag_encoder,City_encoder,city_sub_encoder,State_encoder,k,new_id
0,0,0,1,2015-10-03,2015-10-05,2015-10-05,3,1,0.33,호스텔 유메 노마드,45,3,17,34,6,0,0
3,0,0,3,2015-10-03,2015-10-05,2015-10-06,3,1,0.33,아리마 온천 마을,1152,0,17,35,6,0,1
79,0,0,5,2015-10-03,2015-10-05,2015-10-07,3,1,0.33,스테이크랜드 고베,582,4,17,32,6,0,2
152,0,0,6,2015-10-03,2015-10-05,2015-10-08,3,1,0.33,스타벅스 기타노이진칸점,584,4,17,32,6,0,3
303,0,0,7,2015-10-03,2015-10-05,2015-10-09,3,1,0.33,고베 로코 팜 고베점,625,4,17,32,6,0,4
316,0,0,8,2015-10-03,2015-10-05,2015-10-10,3,1,0.33,고베 모토마치 상점가,391,2,17,32,6,0,5
400,0,0,9,2015-10-03,2015-10-05,2015-10-11,3,1,0.33,난킨마치,1216,0,17,32,6,0,6
511,0,0,10,2015-10-03,2015-10-05,2015-10-12,3,1,0.33,로쇼키,626,4,17,32,6,0,7
520,0,0,11,2015-10-03,2015-10-05,2015-10-13,3,1,0.33,고베 하버랜드 우미에 (구 모자이크),1287,0,17,32,6,0,8
674,0,0,12,2015-10-03,2015-10-05,2015-10-14,3,1,0.33,고베 하버랜드,1288,0,17,32,6,0,9
